# Ontology Visualization

Made by Rita Peixoto

September 2025

In [ ]:
##%pip install owlready2

Imports

In [ ]:
from owlready2 import *
import networkx as nx
import matplotlib.pyplot as plt

Load the ontology file  (change for your file)

In [ ]:
onto = get_ontology("data/Ontology Carros e os seus combostiveis.owx").load()
sync_reasoner()

In [ ]:
def create_onto_graph():
    ontology_graph = nx.DiGraph()

    for cls in onto.classes():
        ontology_graph.add_node(cls.name)

    for cls in onto.classes():
        # Subclass relationships
        for super_cls in cls.is_a:
            if isinstance(super_cls, ThingClass): # Check if it's a class
                ontology_graph.add_edge(cls.name, super_cls.name, relation="is_a")

        # Relationships defined by properties (object properties)
        for restriction in cls.is_a:
            if isinstance(restriction, Restriction):
                if restriction.type == SOME or restriction.type == ONLY or restriction.type == EXACTLY or restriction.type == MIN or restriction.type == MAX or restriction.type == VALUE:
                    if isinstance(restriction.value, ThingClass):
                        ontology_graph.add_edge(cls.name, restriction.value.name, relation=restriction.property.name)
    return ontology_graph

def create_individuals_graph():
    individual_graph = nx.DiGraph()
    for ind in onto.individuals():
        individual_graph.add_node(ind.name)
        for property in ind.get_properties():
            for value in property[ind]:
                if isinstance(value, Thing):
                    individual_graph.add_edge(ind.name, value.name, relation=property.name)
                else: # Add data properties
                    individual_graph.add_edge(ind.name, str(value), relation=property.name)
    return individual_graph

def visualize_graph(graph, title, node_color):
    plt.figure(figsize=(12, 10))
    pos = nx.spring_layout(graph, k=0.8)
    nx.draw_networkx(graph, pos, with_labels=True, node_size=3000, node_color=node_color, font_size=10, width=2, edge_color="gray")
    edge_labels = nx.get_edge_attributes(graph, 'relation')
    nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels, font_color='blue', font_size=10) # Changed font color and size
    plt.title(title)
    plt.axis("off")
    plt.show()


In [ ]:
ontology_graph = create_onto_graph()
individuals_graph = create_individuals_graph()
visualize_graph(ontology_graph, "Ontology Class Relationships", "skyblue")
visualize_graph(individuals_graph, "Ontology Individual Relationships", "lightgreen")